In [21]:
# Loading Relevant Packages 
import torch
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import XLNetConfig, XLNetModel
from transformers import XLNetLMHeadModel, XLNetTokenizer
from tqdm import tqdm

In [22]:
#Loading the dataset 
df = pd.read_csv('news_summary_more.csv', encoding='latin-1', nrows= 1000)
print(df)

                                             headlines  \
0    upGrad learner switches to career in ML & Al w...   
1    Delhi techie wins free food from Swiggy for on...   
2    New Zealand end Rohit Sharma-led India's 12-ma...   
3    Aegon life iTerm insurance plan helps customer...   
4    Have known Hirani for yrs, what if MeToo claim...   
..                                                 ...   
995  Aamir's 'Rubaru Roshni' screened for Sri Sri R...   
996  Playing Meena Thackeray was an honour: Amrita ...   
997  Won't apologise: Kangana on Karni Sena's threa...   
998  Wonderful, humbling feeling: Anupam on meeting...   
999  Erotic drawing by late rapper Tupac sold for $...   

                                                  text  
0    Saurav Kant, an alumnus of upGrad and IIIT-B's...  
1    Kunal Shah's credit card bill payment platform...  
2    New Zealand defeated India by 8 wickets in the...  
3    With Aegon Life iTerm Insurance plan, customer...  
4    Speaking abou

In [4]:
#Checking if the variable distribution is even 
df.count()

headlines    1000
text         1000
dtype: int64

### 'text' Summarizations using BERT using pre-trained model 

In [6]:
texts_to_summarize = df['text']
texts_to_summarize

0      Saurav Kant, an alumnus of upGrad and IIIT-B's...
1      Kunal Shah's credit card bill payment platform...
2      New Zealand defeated India by 8 wickets in the...
3      With Aegon Life iTerm Insurance plan, customer...
4      Speaking about the sexual harassment allegatio...
                             ...                        
995    Aamir Khan screened his upcoming short film 'R...
996    Amrita Rao, who plays Bal Thackeray's wife Mee...
997    Kangana Ranaut, who was threatened by the Karn...
998    Sharing a picture with US talk show host Jimmy...
999    An erotic drawing by late rapper Tupac Shakur ...
Name: text, Length: 1000, dtype: object

In [25]:
model_name = 'facebook/bart-large-cnn'
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

texts_to_summarize=df['text']

# Tokenize and summarize
summaries = []
for index, text_to_summarize in tqdm(texts_to_summarize.items(), total=len(texts_to_summarize), desc="Summarizing"):
    inputs = tokenizer.encode("summarize: " + text_to_summarize, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summaries.append(summary)

Summarizing: 100%|███████████████████████████████████████████████████████████████| 1000/1000 [2:24:21<00:00,  8.66s/it]


In [26]:
if isinstance(df, pd.DataFrame):
    target_column = 'text'
    target_column_index = df.columns.get_loc(target_column)
    df.insert(target_column_index + 1, 'summary-text_BART', summaries)
else:
    print("Error: 'df' is not a DataFrame.")

In [27]:
df.head(10)

,headlines,text,summary-text_BART
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's...","Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer...","With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...,Rajkumar Hirani has been accused by an assista...
5,Rahat Fateh Ali Khan denies getting notice for...,Pakistani singer Rahat Fateh Ali Khan has deni...,Rahat Fateh Ali Khan has denied receiving any ...
6,"India get all out for 92, their lowest ODI tot...",India recorded their lowest ODI total in New Z...,India recorded their lowest ODI total in New Z...
7,Govt directs Alok Verma to join work 1 day bef...,Weeks after ex-CBI Director Alok Verma told th...,The Home Ministry asked him to join work on th...
8,Called PM Modi 'sir' 10 times to satisfy his e...,Andhra Pradesh CM N Chandrababu Naidu has said...,Andhra Pradesh CM N Chandrababu Naidu addresse...
9,"Cong wins Ramgarh bypoll in Rajasthan, takes t...",Congress candidate Shafia Zubair won the Ramga...,Congress candidate Shafia Zubair won the Ramga...


In [ ]:
# Download as feather format (Rdata)
import feather
path = 'BERT_Patents_Similarity_Summary_text.feather'
feather.write_dataframe(df, path)

### 'Text' Summarizations using GPT2

In [12]:
texts_to_summarize_2 = df['text']
texts_to_summarize_2

0      Saurav Kant, an alumnus of upGrad and IIIT-B's...
1      Kunal Shah's credit card bill payment platform...
2      New Zealand defeated India by 8 wickets in the...
3      With Aegon Life iTerm Insurance plan, customer...
4      Speaking about the sexual harassment allegatio...
                             ...                        
995    Aamir Khan screened his upcoming short film 'R...
996    Amrita Rao, who plays Bal Thackeray's wife Mee...
997    Kangana Ranaut, who was threatened by the Karn...
998    Sharing a picture with US talk show host Jimmy...
999    An erotic drawing by late rapper Tupac Shakur ...
Name: text, Length: 1000, dtype: object

In [13]:
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Tokenize and generate text
generated_texts = []
for index, text_to_generate in tqdm(texts_to_summarize_2.items(), total=len(texts_to_summarize_2), desc="Generating Text"):
    inputs = tokenizer.encode(text_to_generate, return_tensors="pt", max_length=1024, truncation=True)
    generated_ids = model.generate(inputs, max_length=150, num_beams=4, early_stopping=True)
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    generated_texts.append(generated_text)

Generating Text:   0%|                                                                        | 0/1000 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:   0%|                                                              | 1/1000 [00:06<1:42:02,  6.13s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:   0%|                                                              | 2/1000 [00:10<1:25:45,  5.16s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attent

Generating Text:   2%|█▎                                                           | 22/1000 [01:56<1:27:28,  5.37s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:   2%|█▍                                                           | 23/1000 [02:02<1:32:02,  5.65s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:   2%|█▍                                                           | 24/1000 [02:07<1:29:50,  5.52s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attent

Generating Text:   4%|██▋                                                          | 44/1000 [03:46<1:08:42,  4.31s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:   4%|██▋                                                          | 45/1000 [03:52<1:17:36,  4.88s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:   5%|██▊                                                          | 46/1000 [03:57<1:18:10,  4.92s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attent

Generating Text:   7%|████                                                         | 66/1000 [05:45<1:24:18,  5.42s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:   7%|████                                                         | 67/1000 [05:49<1:15:51,  4.88s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:   7%|████▏                                                        | 68/1000 [05:53<1:14:25,  4.79s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attent

Generating Text:   9%|█████▎                                                       | 88/1000 [07:39<1:16:26,  5.03s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:   9%|█████▍                                                       | 89/1000 [07:44<1:14:50,  4.93s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:   9%|█████▍                                                       | 90/1000 [07:50<1:18:45,  5.19s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attent

...

Generating Text:  90%|██████████████████████████████████████████████████████      | 902/1000 [1:19:21<09:04,  5.56s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:  90%|██████████████████████████████████████████████████████▏     | 903/1000 [1:19:27<09:24,  5.82s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:  90%|██████████████████████████████████████████████████████▏     | 904/1000 [1:19:33<09:29,  5.93s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attent

Generating Text:  92%|███████████████████████████████████████████████████████▍    | 924/1000 [1:21:19<07:16,  5.74s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:  92%|███████████████████████████████████████████████████████▌    | 925/1000 [1:21:25<07:00,  5.61s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:  93%|███████████████████████████████████████████████████████▌    | 926/1000 [1:21:30<06:44,  5.46s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attent

Generating Text:  95%|████████████████████████████████████████████████████████▊   | 946/1000 [1:23:20<04:37,  5.15s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:  95%|████████████████████████████████████████████████████████▊   | 947/1000 [1:23:27<04:52,  5.52s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:  95%|████████████████████████████████████████████████████████▉   | 948/1000 [1:23:32<04:39,  5.37s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attent

Generating Text:  97%|██████████████████████████████████████████████████████████  | 968/1000 [1:25:18<03:03,  5.74s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:  97%|██████████████████████████████████████████████████████████▏ | 969/1000 [1:25:24<03:02,  5.89s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:  97%|██████████████████████████████████████████████████████████▏ | 970/1000 [1:25:29<02:46,  5.56s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attent

Generating Text:  99%|███████████████████████████████████████████████████████████▍| 990/1000 [1:27:22<00:49,  4.97s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:  99%|███████████████████████████████████████████████████████████▍| 991/1000 [1:27:29<00:49,  5.47s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generating Text:  99%|███████████████████████████████████████████████████████████▌| 992/1000 [1:27:34<00:44,  5.52s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attent

In [28]:
if isinstance(df, pd.DataFrame):
    target_column = 'text'
    target_column_index = df.columns.get_loc(target_column)
    df.insert(target_column_index + 1, 'summary-text_GPT2', generated_texts)
else:
    print("Error: 'df' is not a DataFrame.")

In [29]:
df.head(10)

,headlines,text,summary-text_GPT2,summary-text_BART
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's...","Saurav Kant, an alumnus of upGrad and IIIT-B's...","Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...,Kunal Shah's credit card bill payment platform...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...,New Zealand defeated India by 8 wickets in the...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer...","With Aegon Life iTerm Insurance plan, customer...","With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...,Speaking about the sexual harassment allegatio...,Rajkumar Hirani has been accused by an assista...
5,Rahat Fateh Ali Khan denies getting notice for...,Pakistani singer Rahat Fateh Ali Khan has deni...,Pakistani singer Rahat Fateh Ali Khan has deni...,Rahat Fateh Ali Khan has denied receiving any ...
6,"India get all out for 92, their lowest ODI tot...",India recorded their lowest ODI total in New Z...,India recorded their lowest ODI total in New Z...,India recorded their lowest ODI total in New Z...
7,Govt directs Alok Verma to join work 1 day bef...,Weeks after ex-CBI Director Alok Verma told th...,Weeks after ex-CBI Director Alok Verma told th...,The Home Ministry asked him to join work on th...
8,Called PM Modi 'sir' 10 times to satisfy his e...,Andhra Pradesh CM N Chandrababu Naidu has said...,Andhra Pradesh CM N Chandrababu Naidu has said...,Andhra Pradesh CM N Chandrababu Naidu addresse...
9,"Cong wins Ramgarh bypoll in Rajasthan, takes t...",Congress candidate Shafia Zubair won the Ramga...,Congress candidate Shafia Zubair won the Ramga...,Congress candidate Shafia Zubair won the Ramga...


In [ ]:
# Download as feather format (Rdata)
import feather
path = 'BERT_Patents_Similarity_Summary.feather'
feather.write_dataframe(df, path)

### 'Text' Summarizations using XLNet

In [5]:
texts_to_summarize3 = df['text']

model_name_xlnet = 'xlnet-large-cased'
model_xlnet = XLNetLMHeadModel.from_pretrained(model_name_xlnet)
tokenizer_xlnet = XLNetTokenizer.from_pretrained(model_name_xlnet)

In [6]:
generated_texts_xlnet = []
for index, text_to_generate in tqdm(texts_to_summarize3.items(), total=len(texts_to_summarize3), desc="Generating Text"):
    inputs_xlnet = tokenizer_xlnet.encode(text_to_generate, return_tensors="pt", max_length=1024, truncation=True)
    generated_ids_xlnet = model_xlnet.generate(inputs_xlnet, max_length=150, num_beams=4, early_stopping=True)
    generated_text_xlnet = tokenizer_xlnet.decode(generated_ids_xlnet[0], skip_special_tokens=True)
    generated_texts_xlnet.append(generated_text_xlnet)

Generating Text: 100%|██████████████████████████████████████████████████████████| 1000/1000 [18:47:38<00:00, 67.66s/it]


In [30]:
if isinstance(df, pd.DataFrame):
    target_column = 'text'
    target_column_index = df.columns.get_loc(target_column)
    df.insert(target_column_index + 1, 'summary -text_XLNet', generated_texts_xlnet)
else:
    print("Error: 'df' is not a DataFrame.")

In [31]:
df.head(10)

,headlines,text,summary -text_XLNet,summary-text_GPT2,summary-text_BART
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's...","Saurav Kant, an alumnus of upGrad and IIIT-B's...","Saurav Kant, an alumnus of upGrad and IIIT-B's...","Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...,Kunal Shah's credit card bill payment platform...,Kunal Shah's credit card bill payment platform...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...,New Zealand defeated India by 8 wickets in the...,New Zealand defeated India by 8 wickets in the...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer...","With Aegon Life iTerm Insurance plan, customer...","With Aegon Life iTerm Insurance plan, customer...","With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...,Speaking about the sexual harassment allegatio...,Speaking about the sexual harassment allegatio...,Rajkumar Hirani has been accused by an assista...
5,Rahat Fateh Ali Khan denies getting notice for...,Pakistani singer Rahat Fateh Ali Khan has deni...,Pakistani singer Rahat Fateh Ali Khan has deni...,Pakistani singer Rahat Fateh Ali Khan has deni...,Rahat Fateh Ali Khan has denied receiving any ...
6,"India get all out for 92, their lowest ODI tot...",India recorded their lowest ODI total in New Z...,India recorded their lowest ODI total in New Z...,India recorded their lowest ODI total in New Z...,India recorded their lowest ODI total in New Z...
7,Govt directs Alok Verma to join work 1 day bef...,Weeks after ex-CBI Director Alok Verma told th...,Weeks after ex-CBI Director Alok Verma told th...,Weeks after ex-CBI Director Alok Verma told th...,The Home Ministry asked him to join work on th...
8,Called PM Modi 'sir' 10 times to satisfy his e...,Andhra Pradesh CM N Chandrababu Naidu has said...,Andhra Pradesh CM N Chandrababu Naidu has said...,Andhra Pradesh CM N Chandrababu Naidu has said...,Andhra Pradesh CM N Chandrababu Naidu addresse...
9,"Cong wins Ramgarh bypoll in Rajasthan, takes t...",Congress candidate Shafia Zubair won the Ramga...,Congress candidate Shafia Zubair won the Ramga...,Congress candidate Shafia Zubair won the Ramga...,Congress candidate Shafia Zubair won the Ramga...
